## Cointegration Testing using Engle-Granger

**Functions**

`tsa.coint`

### Exercise 74
Download data on $CAY$ from [Sydney Ludvigson’s site](https://www.sydneyludvigson.com/data-and-appendixes).

### Exercise 75
Is there evidence that these three series are cointegrated in the entire sample?

### Exercise 76
What about in the post-Volker era (start in 1981)?